In [43]:
#call workspace from config file in another environment
from azureml.core import Workspace
ws = Workspace.from_config(path="./.azureml/ws_config.json")

In [44]:
ws.get_details()

{'id': '/subscriptions/818f7123-d5f7-478f-b1ac-78868aa777fe/resourceGroups/DataScience/providers/Microsoft.MachineLearningServices/workspaces/ws_leena_1',
 'name': 'ws_leena_1',
 'location': 'eastus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'sku': 'Basic',
 'workspaceid': '2b3e56d9-9b65-40af-9b04-05b110b53a2b',
 'description': '',
 'friendlyName': 'ws_leena_1',
 'creationTime': '2020-05-14T15:47:26.4118406+00:00',
 'containerRegistry': '/subscriptions/818f7123-d5f7-478f-b1ac-78868aa777fe/resourceGroups/DataScience/providers/Microsoft.ContainerRegistry/registries/wsleena17cab1e48',
 'keyVault': '/subscriptions/818f7123-d5f7-478f-b1ac-78868aa777fe/resourcegroups/datascience/providers/microsoft.keyvault/vaults/wsleena1keyvaultcb29a732',
 'applicationInsights': '/subscriptions/818f7123-d5f7-478f-b1ac-78868aa777fe/resourcegroups/datascience/providers/microsoft.insights/components/wsleena1insights1d9b0934',
 'identityPrincipalId': '2fd02e40-db4f-4972-912d-48f773e32565',
 'i

In [71]:
%%writefile bk_sklearn_lr.py
#Remember our linear regression example? Let's train it locally and register it

#prepare the environment
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import sklearn
#load the data
from sklearn.datasets import load_boston
boston_dataset = load_boston()


#prepare the data
from sklearn.model_selection import train_test_split
num_Rooms_Train, num_Rooms_Test, med_price_Train, med_Price_Test = train_test_split(boston_dataset.data[:,5].reshape(-1,1), boston_dataset.target.reshape(-1,1))


#implement linear regression model
from sklearn.linear_model import LinearRegression
price_room = LinearRegression()
price_room.fit (num_Rooms_Train,med_price_Train)

# Let's save this model fit out
from sklearn.externals import joblib
os.makedirs("outputs", exist_ok=True)
joblib.dump(value=price_room, filename="outputs/bh_lr.pkl")


Writing bk_sklearn_lr.py


In [73]:
#!ls -al
!cat bk_sklearn_lr.py

#Remember our linear regression example? Let's train it locally and register it

#prepare the environment
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import sklearn
#load the data
from sklearn.datasets import load_boston
boston_dataset = load_boston()


#prepare the data
from sklearn.model_selection import train_test_split
num_Rooms_Train, num_Rooms_Test, med_price_Train, med_Price_Test = train_test_split(boston_dataset.data[:,5].reshape(-1,1), boston_dataset.target.reshape(-1,1))


#implement linear regression model
from sklearn.linear_model import LinearRegression
price_room = LinearRegression()
price_room.fit (num_Rooms_Train,med_price_Train)

# Let's save this model fit out
from sklearn.externals import joblib
os.makedirs("outputs", exist_ok=True)
joblib.dump(value=price_room, filename="outputs/bh_lr.pkl")


In [74]:
#Let's register this model with our workspace

from azureml.core.model import Model

model = Model.register(workspace=ws, model_path="outputs/bh_lr.pkl", model_name="boston_housing_lr")

Registering model boston_housing_lr


In [75]:
mymodel = Model(workspace=ws, name="boston_housing_lr")
mymodel.download(exist_ok=True)

'bh_lr.pkl'

In [76]:
from sklearn.externals import joblib
localmodel = joblib.load("bh_lr.pkl")

In [77]:
!ls -al 


total 8
drwxr-xr-x 2 nbuser nbuser     0 Jan  1  1970 .
drwxr-xr-x 1 nbuser nbuser  4096 May 15 14:23 ..
drwxr-xr-x 2 nbuser nbuser     0 May 14 15:48 .azureml
-rw-r--r-- 1 nbuser nbuser   604 May 15  2020 bh_lr.pkl
-rw-r--r-- 1 nbuser nbuser   860 May 15 17:23 bk_sklearn_lr.py
-rw-r--r-- 1 nbuser nbuser 12329 May 15 16:47 create-workspace-sanitized.ipynb
drwxr-xr-x 2 nbuser nbuser     0 May 11 20:19 .ipynb_checkpoints
-rw-r--r-- 1 nbuser nbuser  8563 May 15 16:47 model-deployment-sanitized.ipynb
-rw-r--r-- 1 nbuser nbuser  8831 May 15 17:19 model_training_sanitized.ipynb
-rw-r--r-- 1 nbuser nbuser   860 May 15 17:05 my_sklearn_lr.py
drwxr-xr-x 2 nbuser nbuser     0 May 15 17:05 outputs
-rw-r--r-- 1 nbuser nbuser     0 May 11 20:17 README.md


In [78]:
mymodel.delete()

In [79]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.compute import AmlCompute
list_vms = AmlCompute.supported_vmsizes(workspace=ws)

In [80]:
print(list_vms)

[{'name': 'Standard_DS1_v2', 'vCPUs': 1, 'gpus': 0, 'memoryGB': 3.5, 'maxResourceVolumeMB': 7168}, {'name': 'Standard_DS2_v2', 'vCPUs': 2, 'gpus': 0, 'memoryGB': 7.0, 'maxResourceVolumeMB': 14336}, {'name': 'Standard_DS3_v2', 'vCPUs': 4, 'gpus': 0, 'memoryGB': 14.0, 'maxResourceVolumeMB': 28672}, {'name': 'Standard_DS4_v2', 'vCPUs': 8, 'gpus': 0, 'memoryGB': 28.0, 'maxResourceVolumeMB': 57344}, {'name': 'Standard_DS5_v2', 'vCPUs': 16, 'gpus': 0, 'memoryGB': 56.0, 'maxResourceVolumeMB': 114688}, {'name': 'Standard_DS11_v2', 'vCPUs': 2, 'gpus': 0, 'memoryGB': 14.0, 'maxResourceVolumeMB': 28672}, {'name': 'Standard_DS12_v2', 'vCPUs': 4, 'gpus': 0, 'memoryGB': 28.0, 'maxResourceVolumeMB': 57344}, {'name': 'Standard_DS13_v2', 'vCPUs': 8, 'gpus': 0, 'memoryGB': 56.0, 'maxResourceVolumeMB': 114688}, {'name': 'Standard_DS14_v2', 'vCPUs': 16, 'gpus': 0, 'memoryGB': 112.0, 'maxResourceVolumeMB': 229376}, {'name': 'Standard_DS15_v2', 'vCPUs': 20, 'gpus': 0, 'memoryGB': 140.0, 'maxResourceVolumeMB

In [81]:
compute_config = RunConfiguration()
compute_config.target = "amlcompute"
compute_config.amlcompute.vm_size = "Standard_DS1_v2"

In [82]:
from azureml.core.conda_dependencies import CondaDependencies

dependencies = CondaDependencies()
dependencies.set_python_version("3.6.6")
dependencies.add_pip_package("scikit-learn==0.20.3")
dependencies.add_pip_package("numpy==1.16.2")
dependencies.add_pip_package("pandas==0.23.4")
dependencies.add_pip_package("matplotlib==3.0.0")
compute_config.environment.python.conda_dependencies = dependencies

In [83]:
from azureml.core.experiment import Experiment
from azureml.core import ScriptRunConfig

script_run_config = ScriptRunConfig(source_directory=".",script="bk_sklearn_lr.py", run_config=compute_config)
experiment = Experiment(workspace=ws, name="lq_simple_model")
run = experiment.submit(config=script_run_config)
run.wait_for_completion(show_output=True)

RunId: lq_simple_model_1589563402_dfaad09b
Web View: https://ml.azure.com/experiments/lq_simple_model/runs/lq_simple_model_1589563402_dfaad09b?wsid=/subscriptions/818f7123-d5f7-478f-b1ac-78868aa777fe/resourcegroups/DataScience/workspaces/ws_leena_1

Streaming azureml-logs/55_azureml-execution-tvmps_a89d73527eab6004834b6c5bdc3f775bafcba7be8b0e25dfc702c2bf673cb07d_d.txt

2020-05-15T17:28:30Z Starting output-watcher...
2020-05-15T17:28:30Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
3b1e799c6ebe1c49db0f811ca96016cb7c013070215dae340bcc0c3be6754251

Streaming azureml-logs/65_job_prep-tvmps_a89d73527eab6004834b6c5bdc3f775bafcba7be8b0e25dfc702c2bf673cb07d_d.txt

Entering job preparation. Current time:2020-05-15T17:30:00.006417
Starting job preparation. Current time:2020-05-15T17:30:00.784174
Extracting the control code.
fetching and extracting the control code on master node.
Retrieving project from snapshot: 1ae76c02-4486-4ab5-8603-51b209272323
Starting the daemon thread to

{'runId': 'lq_simple_model_1589563402_dfaad09b',
 'target': 'amlcompute',
 'status': 'Completed',
 'startTimeUtc': '2020-05-15T17:28:25.91896Z',
 'endTimeUtc': '2020-05-15T17:39:33.950058Z',
 'warnings': [{'message': "This compute target type doesn't support non-Docker runs; overriding run configuration to enable Docker.\nPlease enable Docker in the environment section of your run configuration to stop seeing this warning message."}],
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '4feb723c-656a-407b-9f7f-f9969722dba7',
  'AzureML.DerivedImageName': 'azureml/azureml_87eaedd30d35a7d7147fcccc31026c2b',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'runDefinition': {'script': 'bk_sklearn_lr.py',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'amlcompute',
  'dataRef

In [84]:
run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_a89d73527eab6004834b6c5bdc3f775bafcba7be8b0e25dfc702c2bf673cb07d_d.txt',
 'azureml-logs/65_job_prep-tvmps_a89d73527eab6004834b6c5bdc3f775bafcba7be8b0e25dfc702c2bf673cb07d_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_a89d73527eab6004834b6c5bdc3f775bafcba7be8b0e25dfc702c2bf673cb07d_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/97_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/bh_lr.pkl']

In [85]:
run.download_file(name="outputs/bh_lr.pkl")

In [86]:
!ls

bh_lr.pkl			  model-deployment-sanitized.ipynb  outputs
bk_sklearn_lr.py		  model_training_sanitized.ipynb    README.md
create-workspace-sanitized.ipynb  my_sklearn_lr.py


In [87]:
ls -al

total 8
drwxr-xr-x 2 nbuser nbuser     0 Jan  1  1970 ./
drwxr-xr-x 1 nbuser nbuser  4096 May 15 14:23 ../
drwxr-xr-x 2 nbuser nbuser     0 May 14 15:48 .azureml/
-rw-r--r-- 1 nbuser nbuser   604 May 15  2020 bh_lr.pkl
-rw-r--r-- 1 nbuser nbuser   860 May 15 17:23 bk_sklearn_lr.py
-rw-r--r-- 1 nbuser nbuser 12329 May 15 16:47 create-workspace-sanitized.ipynb
drwxr-xr-x 2 nbuser nbuser     0 May 11 20:19 .ipynb_checkpoints/
-rw-r--r-- 1 nbuser nbuser  8563 May 15 16:47 model-deployment-sanitized.ipynb
-rw-r--r-- 1 nbuser nbuser 22905 May 15 17:31 model_training_sanitized.ipynb
-rw-r--r-- 1 nbuser nbuser   860 May 15 17:05 my_sklearn_lr.py
drwxr-xr-x 2 nbuser nbuser     0 May 15 17:05 outputs/
-rw-r--r-- 1 nbuser nbuser     0 May 11 20:17 README.md


In [88]:
from sklearn.externals import joblib
import numpy as np

run_model = joblib.load('bh_lr.pkl')
run_model.predict(np.array([7.354]).reshape(-1,1))

array([[32.5753084]])

In [89]:
from azureml.core.model import Model

model = Model.register(workspace=ws, model_path="bh_lr.pkl", model_name="lq_simple_train")

Registering model lq_simple_train


In [90]:
run.register_model(model_name="sklearn_bh_lr", model_path="outputs/bh_lr.pkl")

Model(workspace=Workspace.create(name='ws_leena_1', subscription_id='818f7123-d5f7-478f-b1ac-78868aa777fe', resource_group='DataScience'), name=sklearn_bh_lr, id=sklearn_bh_lr:1, version=1, tags={}, properties={})